In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
import requests
import os 
import json
import sys 
import re
import time

###### Import selenium dependencies
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.keys import Keys

%matplotlib inline

#### Set up chrome options for the browser

In this section, we can add keyboard arguments to the chrome browser as options. The website https://peter.sh/experiments/chromium-command-line-switches/ contains all the possible extensions to the chrome option class. 

In [2]:
## Set up chrome Options 

chrome_options = Options()  
chrome_options.add_argument("--headless") 

#### Spider Class

In [3]:
class hoaxSpider():
    
    def __init__(self):
        
        self.URL="http://www.warriorelihoax.com"
        self.content = self.get_request()
        self.soup = self.get_soup()
        
    def get_request(self):
        content = requests.get(self.URL).content
        return content
    
    def get_soup(self):
        soup = BeautifulSoup(self.content,'html.parser')
        return soup
    
    def get_names_cases(self):
        all_tables = self.soup.find_all(id = "cat")
        txt = ""
        for i in all_tables:
            txt+=i.text.strip()
        cases = []
        for i in txt.split("\n"):
            reg = re.findall(r"^[^\(]+", i)
            cases.append(reg)
        flat_list = [item for sublist in cases for item in sublist]
        newL = []
        for i in flat_list:
            newL.append(i.replace(u'\xa0', u''))
        return newL[1:] 
    
    def select_hoax_case(self, dropdown = '//*[@id="cat"]'):
        
        self.driver = webdriver.Chrome()
        self.driver.get("http://www.warriorelihoax.com")
        python_button = self.driver.find_element_by_xpath(dropdown + '/option[5]')
        python_button.click()

    def scrape_images(self, dropdown = '//*[@id="cat"]'):
        self.driver = webdriver.Chrome()
        self.driver.get("http://www.warriorelihoax.com")
        option = 20
        while option:
            time.sleep(1)
            try:
                python_button = self.driver.find_element_by_xpath(dropdown + '/option[' + str(option) + ']')
            except:
                break
            python_button.click()
            option += 1
        

In [4]:
spider = hoaxSpider()

In [5]:
case_names = spider.get_names_cases()

In [6]:
spider.scrape_images()

In [7]:
driver = webdriver.Chrome()
driver.get("http://www.warriorelihoax.com")

def scrape_cases_text():
    
    URL="http://www.warriorelihoax.com"
    content=requests.get(URL).content
    soup=BeautifulSoup(content,'html.parser')
    all_tables=soup.find_all(id = "cat")
    txt = ""
    for i in all_tables:
        txt+=i.text.strip()
    cases = []
    for i in txt.split("\n"):
        reg = re.findall(r"^[^\(]+", i)
        cases.append(reg)
    flat_list = [item for sublist in cases for item in sublist]
    newL = []
    for i in flat_list:
        newL.append(i.replace(u'\xa0', u''))
    return newL[1:] 
    
    

def select_hoax_case(dropdown = '//*[@id="cat"]'):
    
    python_button = driver.find_element_by_xpath(dropdown + '/option[2]')
    python_button.click()
    
    
def dowload_images()
    
select_hoax_case()

SyntaxError: invalid syntax (<ipython-input-7-b16e5dc1797c>, line 31)